# Diabetic data  (methods of handling Imbalanced data)

In [1]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, confusion_matrix
import csv
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


# 1 Data Exploring
The data that is used in this project originally comes from the UCI machine learning repository (https://archive.ics.uci.edu/ml/datasets/diabetes+130-us+hospitals+for+years+1999-2008). The data consists of over 100000 hospital admissions from patients with diabetes from 130 US hospitals between 1999-2008.

In this part, We tried to explore the following things:
* The columns and records of the data
* The data type of each column
* Whether there is missing values such as null or NAs
* The unique values and their counts of each columns
* The ratio between the number of readmitted records and non-readmitted records

In [2]:
data = pd.read_csv('diabetic_data.csv')
data.shape

(101766, 50)

In [3]:
data

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,AfricanAmerican,Male,[70-80),?,1,3,7,3,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,443847782,74694222,AfricanAmerican,Female,[80-90),?,1,4,5,5,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,443854148,41088789,Caucasian,Male,[70-80),?,1,1,7,1,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,443857166,31693671,Caucasian,Female,[80-90),?,2,3,7,10,...,No,Up,No,No,No,No,No,Ch,Yes,NO


In [4]:
data.dtypes

encounter_id                 int64
patient_nbr                  int64
race                        object
gender                      object
age                         object
weight                      object
admission_type_id            int64
discharge_disposition_id     int64
admission_source_id          int64
time_in_hospital             int64
payer_code                  object
medical_specialty           object
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
diag_1                      object
diag_2                      object
diag_3                      object
number_diagnoses             int64
max_glu_serum               object
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride         

In [5]:
data.isnull().sum()

encounter_id                0
patient_nbr                 0
race                        0
gender                      0
age                         0
weight                      0
admission_type_id           0
discharge_disposition_id    0
admission_source_id         0
time_in_hospital            0
payer_code                  0
medical_specialty           0
num_lab_procedures          0
num_procedures              0
num_medications             0
number_outpatient           0
number_emergency            0
number_inpatient            0
diag_1                      0
diag_2                      0
diag_3                      0
number_diagnoses            0
max_glu_serum               0
A1Cresult                   0
metformin                   0
repaglinide                 0
nateglinide                 0
chlorpropamide              0
glimepiride                 0
acetohexamide               0
glipizide                   0
glyburide                   0
tolbutamide                 0
pioglitazo

In [6]:
data.isna().sum()

encounter_id                0
patient_nbr                 0
race                        0
gender                      0
age                         0
weight                      0
admission_type_id           0
discharge_disposition_id    0
admission_source_id         0
time_in_hospital            0
payer_code                  0
medical_specialty           0
num_lab_procedures          0
num_procedures              0
num_medications             0
number_outpatient           0
number_emergency            0
number_inpatient            0
diag_1                      0
diag_2                      0
diag_3                      0
number_diagnoses            0
max_glu_serum               0
A1Cresult                   0
metformin                   0
repaglinide                 0
nateglinide                 0
chlorpropamide              0
glimepiride                 0
acetohexamide               0
glipizide                   0
glyburide                   0
tolbutamide                 0
pioglitazo

In [7]:
for i in data.columns:
    print(data[i].value_counts())
    print("-----------------------------------------")

96210942     1
89943846     1
384306986    1
94650156     1
83156784     1
            ..
74454612     1
208073976    1
166229592    1
38340702     1
77856768     1
Name: encounter_id, Length: 101766, dtype: int64
-----------------------------------------
88785891     40
43140906     28
23199021     23
1660293      23
88227540     23
             ..
71081460      1
30060018      1
67443444      1
141344240     1
93251151      1
Name: patient_nbr, Length: 71518, dtype: int64
-----------------------------------------
Caucasian          76099
AfricanAmerican    19210
?                   2273
Hispanic            2037
Other               1506
Asian                641
Name: race, dtype: int64
-----------------------------------------
Female             54708
Male               47055
Unknown/Invalid        3
Name: gender, dtype: int64
-----------------------------------------
[70-80)     26068
[60-70)     22483
[50-60)     17256
[80-90)     17197
[40-50)      9685
[30-40)      3775
[90-100)  

There are no null values or NAs but lots "?" and little "unknown" in the data. 
* race: ?(2273)   
* gender: Unknown/Invalid(3)
* weight: ?(98569)
* payer_code: ?(40256)  
* medical_specialty: ?(49949)

In [8]:
len(data[data['readmitted']=='<30'])/len(data[data['readmitted']!='<30'])

0.1256180247541727

# 2 Data Cleaning
In this section, we drop some non-influencial columns and some sepecial records, deal with missing values and transform the format or data type of some columns.

In [9]:
df_cleaning = data.copy()

## 2.1 Drop non-influence columns
Drop columns that have same value with every records

In [10]:
# examide and citoglipton only have 1 value, have no influence to output, drop
df_cleaning.drop(["examide", "citoglipton"], axis = 1, inplace = True)

## 2.2 Drop special records
Drop records according to the value of discharge_disposition_id, which tells us where the patient went after the hospitalization. If we look at the IDs_mapping.csv we can see that 11,13,14,19,20,21 are related to death or hospice. We should remove these samples from the predictive model.

In [11]:
df_cleaning.groupby('discharge_disposition_id').size()

discharge_disposition_id
1     60234
2      2128
3     13954
4       815
5      1184
6     12902
7       623
8       108
9        21
10        6
11     1642
12        3
13      399
14      372
15       63
16       11
17       14
18     3691
19        8
20        2
22     1993
23      412
24       48
25      989
27        5
28      139
dtype: int64

In [12]:
df_cleaning = df_cleaning.loc[~data.discharge_disposition_id.isin([11,13,14,19,20,21])]

## 2.3 Deal with missing values

Deal with weight missing values

In [13]:
df_cleaning.groupby('weight').size().sort_values(ascending = False)

weight
?            96218
[75-100)      1312
[50-75)        867
[100-125)      617
[125-150)      143
[25-50)         90
[0-25)          48
[150-175)       34
[175-200)       11
>200             3
dtype: int64

In [14]:
# too much ? in weight, and other features have no help to fill in the missing value in weight,
# so transform the weight feature into binary.
df_cleaning.weight[df_cleaning['weight']!='?'] = "1"
df_cleaning.weight[df_cleaning['weight']=='?'] = "0"

Drop records with "unknown" value in gender.

In [15]:
# A little unknown in gender, so just drop
df_cleaning = df_cleaning[df_cleaning['gender']!= 'Unknown/Invalid']

Regard "?" in other columns as a kind of category.

## 2.4 Change the data type and format of some columns

In [16]:
# Change the type of ids
ids_col = ['admission_type_id', 'discharge_disposition_id', 'admission_source_id']
df_cleaning[ids_col] = df_cleaning[ids_col].astype('str')

In [17]:
# map age
age_map = {'[0-10)':5, 
          '[10-20)':15, 
          '[20-30)':25, 
          '[30-40)':35, 
          '[40-50)':45, 
          '[50-60)':55,
          '[60-70)':65, 
          '[70-80)':75, 
          '[80-90)':85, 
          '[90-100)':95}
df_cleaning['age'] = df_cleaning['age'].replace(age_map)

In [18]:
# Deal with output values
df_cleaning.readmitted[df_cleaning['readmitted'] != '<30'] = 0
df_cleaning.readmitted[df_cleaning['readmitted'] == '<30'] = 1

## 2.5 Check the data types after cleanig

In [19]:
df_cleaning.dtypes

encounter_id                 int64
patient_nbr                  int64
race                        object
gender                      object
age                          int64
weight                      object
admission_type_id           object
discharge_disposition_id    object
admission_source_id         object
time_in_hospital             int64
payer_code                  object
medical_specialty           object
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
diag_1                      object
diag_2                      object
diag_3                      object
number_diagnoses             int64
max_glu_serum               object
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride         

# 3 Feature Engeering
In this section, we create features for our predictive model.

In [20]:
df_feature = df_cleaning.copy()

In [21]:
df_feature.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

To turn these non-numerical data into variables, the simplest thing is to use a technique called one-hot encoding, which will be explained below. 

In [22]:
df_feature.groupby('medical_specialty').size().sort_values(ascending = False)

medical_specialty
?                                48614
InternalMedicine                 14237
Emergency/Trauma                  7419
Family/GeneralPractice            7252
Cardiology                        5278
                                 ...  
Speech                               1
Pediatrics-InfectiousDiseases        1
SportsMedicine                       1
Proctology                           1
Neurophysiology                      1
Length: 73, dtype: int64

We can see that most of them are unknown and that the count drops off pretty quickly. We don't want to add 73 new variables since some of them only have a few samples. As an alternative, we can create a new variable that only has 11 options (the top 10 specialities and then an other category). Obviously, there are other options for bucketing, but this is one of the easiest methods.

In [23]:
top_10 = ['?','InternalMedicine','Emergency/Trauma','Family/GeneralPractice', 'Cardiology','Surgery-General' ,
          'Nephrology','Orthopedics', 'Orthopedics-Reconstructive','Radiologist']

# make a new column with duplicated data
df_feature['med_spec'] = df_feature['medical_specialty'].copy()

# replace all specialties not in top 10 with 'Other' category
df_feature.loc[~df_feature.med_spec.isin(top_10),'med_spec'] = 'Other'

 In one-hot encoding, you create a new column for each unique value in that column. Then the value of the column is 1 if the sample has that unique value or 0 otherwise. For example, for the column race, we would create new columns ('race_Caucasian','race_AfricanAmerican', etc). If the patient's race is Caucasian, the patient gets a 1 under 'race_Caucasian' and 0 under the rest of the race columns. To create these one-hot encoding columns, we can use the get_dummies function. Now the problem is that if we create a column for each unique value, we have correlated columns. In other words, the value in one column can be figured out by looking at the rest of the columns. For example, if the sample is not AfricanAmerican, Asian, Causasian, Hispance or Other, it must be "?". To deal with this, we can use the drop_first option, which will drop the first categorical value for each column.

In [24]:
# Split column names into numerical and category, in order to get_dummies of category features.
numerical = ['age', 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 
            'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']

category = ['race', 'gender', 'weight','admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'max_glu_serum', 'A1Cresult', 'metformin','repaglinide',
        'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'insulin',
        'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed','payer_code']
# We don't include diag1, diag2, diag3 - which are categorical and have a lot of values. 
# We will not use these as part of this project, but could group these ICD codes to reduce the dimension. 
# We will use number_diagnoses to capture some of this information. 

In [25]:
dfcate = pd.get_dummies(df_feature[category + ['med_spec']],drop_first = True)
dfcate.shape

(99340, 133)

In [26]:
dfnum = df_feature[numerical]

In [27]:
df_X_all = pd.concat([dfnum,dfcate], axis = 1)

# 4 Split and Balance the data
Imbalanced data refers to a situation where the number of observations is not the same for all the classes in a classification dataset. Most machine learning algorithms work best when the number of samples in each class are about equal. Given that these algorithms aim to minimize the overall error rate, instead of paying special attention to the minority class, they may fail to make an accurate prediction for this class if they don’t get the necessary amount of information about it. Let’s consider an even more extreme example than our breast cancer dataset: assume we had 10 malignant vs 90 benign samples. A machine learning model that has been trained and tested on such a dataset could now predict “benign” for all samples and still gain a very high accuracy. An unbalanced dataset will bias the prediction model towards the more common class! Thus, we need to balance the data and use the right evaluation metrics.

In [28]:
X = df_X_all.copy()
X_columns = X.columns
y = df_feature['readmitted'].astype('int')
df_data = pd.concat([X,y], axis = 1)

In [29]:
def calc_prevalence(y_actual):
    return (sum(y_actual)/len(y_actual))

In [30]:
# prevalence before split
calc_prevalence(df_data['readmitted'].values)

0.1138916851218039

## 4.1 Split data

In this project, we will split into 80% train, 20% test.  We can use "sample" to extract 20% (using "frac") of the data to be used for test splits.

In [31]:
# shuffle the samples
df_data = df_data.sample(n = len(df_data), random_state = 42)
df_data = df_data.reset_index(drop = True)

# Save 20% of the data as test data 
df_test = df_data.sample(frac = 0.2,random_state = 42)

# use the rest of the data as training data
df_train = df_data.drop(df_test.index)
df_train = df_train.sample(n = len(df_train), random_state = 42).reset_index(drop = True)

# check the prevalence of train and test
print('Test prevalence(n = %d):%.3f'%(len(df_test),calc_prevalence(df_test.readmitted.values)))
print('Train all prevalence(n = %d):%.3f'%(len(df_train), calc_prevalence(df_train.readmitted.values)))

Test prevalence(n = 19868):0.114
Train all prevalence(n = 79472):0.114


## 4.2 Balance data
We only balance the train data and keep the test untouched to ensure that the test set reflects the reality.

### 4.2.1 Under-sampling
Under-sampling balances the dataset by reducing the size of the abundant class. This method is used when quantity of data is sufficient. By keeping all samples in the rare class and randomly selecting an equal number of samples in the abundant class, a balanced new dataset can be retrieved for further modelling.

In [32]:
def under_sample(df_train):
    rare_df = df_train[df_train['readmitted']== 1]
    abundant_df = df_train.drop(rare_df.index)
    under_df = abundant_df.sample(n = len(rare_df), random_state = 1)
    df_train_under = pd.concat([rare_df,under_df], axis = 0)
    return df_train_under

### 4.2.2 Over-sampling
On the contrary, oversampling is used when the quantity of data is insufficient. It tries to balance dataset by increasing the size of rare samples. Rather than getting rid of abundant samples, new rare samples are generated by using e.g. bootstrapping or SMOTE (Synthetic Minority Over-Sampling Technique) 

#### a) bootstrapping
If we re-sample records with replacement from our data, we can treat the re-sampled dataset as a new dataset we collected in a parallel universe.

In [33]:
def bootstrapping(df_train):
    abundant_df = df_train[df_train['readmitted']== 0]
    rare_df = df_train[df_train['readmitted']== 1]
    temp_rare = rare_df.sample(n = len(abundant_df), random_state = 1, replace=True)
    df_train_bootstrapping = pd.concat([temp_rare,abundant_df], axis = 0)
    return df_train_bootstrapping

#### b) SMOTE (Synthetic Minority Over-Sampling Technique)
This technique was described by Nitesh Chawla, et al. in their 2002 paper named for the technique titled “SMOTE: Synthetic Minority Over-sampling Technique.”

SMOTE works by selecting examples that are close in the feature space, drawing a line between the examples in the feature space and drawing a new sample at a point along that line.

Specifically, a random example from the minority class is first chosen. Then k of the nearest neighbors for that example are found (typically k=5). A randomly selected neighbor is chosen and a synthetic example is created at a randomly selected point between the two examples in feature space.

In [34]:
def smote(df_train):
    smo = SMOTE(random_state=42)
    X = df_train[X_columns]
    y = df_train["readmitted"]
    X_smo, y_smo = smo.fit_sample(X, y)
    df_train_smote = pd.concat([X_smo, y_smo], axis = 1)
    return df_train_smote

### 4.2.3 Ensemble different resampled datasets
The easiest way to successfully generalize a model is by using more data. The problem is that out-of-the-box classifiers like logistic regression or random forest tend to generalize by discarding the rare class. One easy best practice is building n models that use all the samples of the rare class and n-differing samples of the abundant class. Given that you want to ensemble 10 models, you would keep e.g. the 1,000 cases of the rare class and randomly sample 10,000 cases of the abundant class. Then you just split the 10,000 cases in 10 chunks and train 10 different models. This approach is simple and perfectly horizontally scalable if you have a lot of data, since you can just train and run your models on different cluster nodes. Ensemble models also tend to generalize better, which makes this approach easy to handle.


In [35]:
def ensemble(df_train):
    abundant_df = df_train[df_train['readmitted']== 0]
    rare_df = df_train[df_train['readmitted']== 1]
    abundant_df = abundant_df.sample(n = len(abundant_df), random_state = 42)    # shuffle the data
    abundant_df = abundant_df.reset_index(drop = True)
    n = len(abundant_df)//5
    df_train_ensemble1 = pd.concat([abundant_df[:n], rare_df], axis = 0)
    df_train_ensemble2 = pd.concat([abundant_df[n:(n*2)], rare_df], axis = 0)
    df_train_ensemble3 = pd.concat([abundant_df[(n*2):(n*3)], rare_df], axis = 0)
    df_train_ensemble4 = pd.concat([abundant_df[(n*3):(n*4)], rare_df], axis = 0)
    df_train_ensemble5 = pd.concat([abundant_df[(n*4):], rare_df], axis = 0)
    return [df_train_ensemble1, df_train_ensemble2, df_train_ensemble3, df_train_ensemble4, df_train_ensemble5]

In [36]:
data_list = ensemble(df_train)

# 5 Algorithms
 We use cross validation in train set to tune the parameters and select models.

In [37]:
def report(y_actual, y_pred, thresh):    
    auc = roc_auc_score(y_actual, y_pred)
    accuracy = accuracy_score(y_actual, (y_pred > thresh))
    cm = confusion_matrix(y_actual, (y_pred > thresh))
    tn, fp, fn, tp = confusion_matrix(y_actual, (y_pred > thresh)).ravel()
    recall = recall_score(y_actual, (y_pred > thresh))
    precision = precision_score(y_actual, (y_pred > thresh))
    specificity = sum((y_pred < thresh) & (y_actual == 0)) /sum(y_actual ==0)
    TPR = tp/(tp+fn)
    FNR = fn/(tp+fn)
    f1 = (2*recall*precision)/(recall+precision)
    return auc, accuracy, TPR, FNR, recall, precision, specificity, calc_prevalence(y_actual), f1

In [38]:
# scale data
def scale_data(X_train,X_test):
    scaler  = StandardScaler()
    scaler.fit(X_train)
    X_train_tf = scaler.transform(X_train)
    X_test_tf = scaler.transform(X_test)
    return X_train_tf,X_test_tf

In [39]:
# cross validation
def cross_val(df_train, name, clf, balance_method):
    sfolder = StratifiedKFold(n_splits = 5,random_state =1,shuffle=False)
    all_columns = df_train.columns
    X = df_train[X_columns].values
    y = df_train["readmitted"].values
    accuracy_list, auc_list, TPR_list, FNR_list, recall_list, precision_list, specificity_list, prevalence_list, f1_list  = [], [], [], [], [], [], [], [], []
    for train_index, test_index in sfolder.split(X,y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        cv_train = pd.concat([pd.DataFrame(X_train), pd.DataFrame(y_train)], axis = 1)
        cv_train.columns = all_columns
        
        # Balance data
        if balance_method == "under_sample":
            cv_train_B = under_sample(cv_train) 
        if balance_method == "bootstrapping":
            cv_train_B = bootstrapping(cv_train)
        if balance_method == "SMOTE":
            cv_train_B = smote(cv_train)
        
        # scale data
        X_train_tf, X_test_tf = scale_data(cv_train_B[X_columns].values,X_test)
        y_train_B = cv_train_B["readmitted"]
        
        # fit model
        clf.fit(X_train_tf, y_train_B)
        y_test_preds = clf.predict_proba(X_test_tf)[:,1]
        auc, accuracy, TPR, FNR, recall, precision, specificity, prevalence, f1 = report(y_test, y_test_preds, thresh)
        accuracy_list.append(accuracy)
        auc_list.append(auc)
        TPR_list.append(TPR)
        FNR_list.append(FNR)
        recall_list.append(recall)
        precision_list.append(precision)
        specificity_list.append(specificity)
        prevalence_list.append(prevalence)
        f1_list.append(f1)

    accuracy_mean, accuracy_std = np.mean(accuracy_list), np.std(accuracy_list)
    auc_mean, auc_std = np.mean(auc_list), np.std(auc_list)
    TPR_mean, TPR_std = np.mean(TPR_list), np.std(TPR_list)
    FNR_mean, FNR_std = np.mean(FNR_list), np.std(FNR_list)
    recall_mean, precision_mean = np.mean(recall_list), np.mean(precision_list)
    specificity_mean, prevalence_mean, f1_mean = np.mean(specificity_list), np.mean(prevalence_list), np.mean(f1_list)
    
    print('name:', name) 
    print('auc_mean:%.3f'%auc_mean, 'auc_std:%.3f'%auc_std, 'accuracy_mean:%.3f'%accuracy_mean, 'accuracy_std:%.3f'%accuracy_std)
    print('TPR_mean:%.3f'%TPR_mean, 'TPR_std:%.3f'%TPR_std, 'FNR_mean:%.3f'%FNR_mean, 'FNR_std:%.3f'%FNR_std)
    print('recall_mean:%.3f'%recall_mean)
    print('precision_mean:%.3f'%precision_mean)
    print('specificity_mean:%.3f'%specificity_mean)
    print('prevalence_mean:%.3f'%prevalence_mean)
    print('f1_mean:%.3f'%f1_mean)
    print('  ')
    return accuracy_mean, accuracy_std, auc_mean, auc_std, TPR_mean, TPR_std, FNR_mean, FNR_std

In [40]:
def ensemble_predict(clf_list,data_list,X_test,y_test):
    accuracy_list, auc_list, TPR_list, FNR_list, recall_list, precision_list, specificity_list, prevalence_list, f1_list  = [], [], [], [], [], [], [], [], []
    for i in range(len(data_list)):
        data = data_list[i]
        X_train_tf, X_test_tf = scale_data(data[X_columns].values,X_test)
        y_train = data["readmitted"].values
        clf = clf_list[i]
        clf.fit(X_train_tf, y_train)
        y_test_preds = clf.predict_proba(X_test_tf)[:,1]
        auc, accuracy, TPR, FNR, recall, precision, specificity, prevalence, f1 = report(y_test, y_test_preds, thresh)
        accuracy_list.append(accuracy)
        auc_list.append(auc)
        TPR_list.append(TPR)
        FNR_list.append(FNR)
        recall_list.append(recall)
        precision_list.append(precision)
        specificity_list.append(specificity)
        prevalence_list.append(prevalence)
        f1_list.append(f1)
    print('auc_mean:%.3f'%np.mean(auc_list), 'accuracy_mean:%.3f'%np.mean(accuracy_list))
    print('TPR_mean:%.3f'%np.mean(TPR_list), 'FNR_mean:%.3f'%np.mean(FNR_list))
    print('recall_mean:%.3f'%np.mean(recall_list))
    print('precision_mean:%.3f'%np.mean(precision_list))
    print('specificity_mean:%.3f'%np.mean(specificity_list))
    print('prevalence_mean:%.3f'%np.mean(prevalence_list))
    print('f1_mean:%.3f'%np.mean(f1_list))
    print('  ')
    return np.mean(auc_list),np.mean(accuracy_list),np.mean(TPR_list),np.mean(FNR_list)

In [41]:
thresh = 0.5

## 5.1 Logistic regression

In [42]:
lr = LogisticRegression(random_state = 42)

## 5.2 Decision Tree

In [43]:
dt = DecisionTreeClassifier(max_depth = 10, random_state = 42)

## 5.3 Random Forest

In [44]:
rf = RandomForestClassifier(max_depth = 6, random_state = 42)

## 5.4 SGDClassifier

In [45]:
sgdc = SGDClassifier(loss = 'log',alpha = 0.1,random_state = 42)

## 5.5 GradientBoostingClassifier

In [46]:
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=3, random_state=42)

# 6 Model Selection

In [47]:
# Generate a file of balance methods comparion while selecting best-performenced model
with open('Balance methods comparison of Imbalance data.csv','w') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(['methods','','Logistic Regression','','','','Decision Tree','','','','Random Forest','','','','SGDClassifier','','','','GradientBoostingClassifier','','',''])
    names = ["under_sample", "bootstrapping", "SMOTE"]
    for i in range(len(names)):
        print("Balance method:", names[i])
        print(" ")
        lr_accuracy_mean, lr_accuracy_std, lr_auc_mean, lr_auc_std, lr_TPR_mean, lr_TPR_std, lr_FNR_mean, lr_FNR_std = cross_val(df_train, 'Logistic Regression', lr, names[i])        
        dt_accuracy_mean, dt_accuracy_std, dt_auc_mean, dt_auc_std, dt_TPR_mean, dt_TPR_std, dt_FNR_mean, dt_FNR_std = cross_val(df_train, 'Decision Tree', dt, names[i])
        rf_accuracy_mean, rf_accuracy_std, rf_auc_mean, rf_auc_std, rf_TPR_mean, rf_TPR_std,rf_FNR_mean, rf_FNR_std = cross_val(df_train, 'Random Forest', rf, names[i])
        sgdc_accuracy_mean, sgdc_accuracy_std, sgdc_auc_mean, sgdc_auc_std, sgdc_TPR_mean, sgdc_TPR_std, sgdc_FNR_mean, sgdc_FNR_std = cross_val(df_train, 'SGDClassifier', sgdc, names[i])
        gbc_accuracy_mean, gbc_accuracy_std, gbc_auc_mean, gbc_auc_std, gbc_TPR_mean, gbc_TPR_std,gbc_FNR_mean, gbc_FNR_std = cross_val(df_train, 'GradientBoostingClassifier', gbc, names[i])
        f_csv.writerow([names[i],'',
                        'lr_accuracy_mean','lr_auc_mean','lr_TPR_mean','lr_FNR_mean',
                        'dt_accuracy_mean','dt_auc_mean','dt_TPR_mean','dt_FNR_mean',
                        'rf_accuracy_mean','rf_auc_mean','rf_TPR_mean','rf_FNR_mean',
                        'sgdc_accuracy_mean','sgdc_auc_mean','sgdc_TPR_mean','sgdc_FNR_mean',
                        'gbc_accuracy_mean','gbc_auc_mean','gbc_TPR_mean','gbc_FNR_mean'])
        f_csv.writerow(['','',lr_accuracy_mean,lr_auc_mean,lr_TPR_mean,lr_FNR_mean,
                       dt_accuracy_mean,dt_auc_mean,dt_TPR_mean,dt_FNR_mean,
                        rf_accuracy_mean,rf_auc_mean,rf_TPR_mean,rf_FNR_mean,
                        sgdc_accuracy_mean,sgdc_auc_mean,sgdc_TPR_mean,sgdc_FNR_mean,
                        gbc_accuracy_mean,gbc_auc_mean,gbc_TPR_mean,gbc_FNR_mean])
        f_csv.writerow(['','','lr_accuracy_std','lr_auc_std','lr_TPR_std','lr_FNR_std',
                        'dt_accuracy_std','dt_auc_std','dt_TPR_std','dt_FNR_std',
                        'rf_accuracy_std','rf_auc_std','rf_TPR_std','rf_FNR_std',
                        'sgdc_accuracy_std','sgdc_auc_std','sgdc_TPR_std','sgdc_FNR_std',
                        'gbc_accuracy_std','gbc_auc_std','gbc_TPR_std','gbc_FNR_std'])
        f_csv.writerow(['','',lr_accuracy_std,lr_auc_std,lr_TPR_std,lr_FNR_std,
                       dt_accuracy_std,dt_auc_std,dt_TPR_std,dt_FNR_std,
                        rf_accuracy_std,rf_auc_std,rf_TPR_std,rf_FNR_std,
                        sgdc_accuracy_std,sgdc_auc_std,sgdc_TPR_std,sgdc_FNR_std,
                        gbc_accuracy_std,gbc_auc_std,gbc_TPR_std,gbc_FNR_std])
        print("The average auc of",names[i],":", (lr_auc_mean+dt_auc_mean+rf_auc_mean+sgdc_auc_mean+gbc_auc_mean)/5)
        print("--------------------------------------------------------")
    X_test = df_test[X_columns].values
    y_test = df_test['readmitted'].values
    print("Balance method: Ensemble different models")
    auc_mean,accuracy_mean,TPR_mean,FNR_mean = ensemble_predict([lr,dt,rf,sgdc,gbc],data_list,X_test,y_test)
    f_csv.writerow(["Ensemble",'','accuracy_mean','auc_mean','TPR_mean','FNR_mean'])
    f_csv.writerow(['','',accuracy_mean,auc_mean,TPR_mean,FNR_mean])

Balance method: under_sample
 
name: Logistic Regression
auc_mean:0.656 auc_std:0.005 accuracy_mean:0.662 accuracy_std:0.005
TPR_mean:0.540 TPR_std:0.006 FNR_mean:0.460 FNR_std:0.006
recall_mean:0.540
precision_mean:0.177
specificity_mean:0.678
prevalence_mean:0.114
f1_mean:0.266
  
name: Decision Tree
auc_mean:0.622 auc_std:0.004 accuracy_mean:0.631 accuracy_std:0.009
TPR_mean:0.558 TPR_std:0.015 FNR_mean:0.442 FNR_std:0.015
recall_mean:0.558
precision_mean:0.166
specificity_mean:0.637
prevalence_mean:0.114
f1_mean:0.256
  
name: Random Forest
auc_mean:0.655 auc_std:0.005 accuracy_mean:0.628 accuracy_std:0.014
TPR_mean:0.593 TPR_std:0.013 FNR_mean:0.407 FNR_std:0.013
recall_mean:0.593
precision_mean:0.172
specificity_mean:0.633
prevalence_mean:0.114
f1_mean:0.267
  
name: SGDClassifier
auc_mean:0.654 auc_std:0.006 accuracy_mean:0.654 accuracy_std:0.007
TPR_mean:0.549 TPR_std:0.008 FNR_mean:0.451 FNR_std:0.008
recall_mean:0.549
precision_mean:0.175
specificity_mean:0.668
prevalence_mea

In this project, we use AUC as the criterion:

The Bootstrapping method performs best. Logistic Regression performs best.

In [50]:
train_B = bootstrapping(df_train)    # Balance the whole train set
X_train_B = train_B[X_columns].values
y_train_B = train_B["readmitted"].values
X_train_tf,X_test_tf = scale_data(X_train_B,X_test)    # scale whole train set and untouched test
lr.fit(X_train_tf, y_train_B)
y_train_preds = lr.predict_proba(X_train_tf)[:,1]
y_test_preds = lr.predict_proba(X_test_tf)[:,1]

In [51]:
print('Training:')
auc, accuracy, TPR, FNR, recall, precision, specificity, prevalence, f1 = report(y_train_B, y_train_preds, thresh)
print('auc:%.3f'%auc,'accuracy:%.3f'%accuracy)
print('TPR:%.3f'%TPR,'FNR:%.3f'%FNR)
print('recall:%.3f'%recall)
print('precision:%.3f'%precision)
print('specificity:%.3f'%specificity)
print('prevalence:%.3f'%prevalence)
print('f1:%.3f'%f1)
print('  ')
print('Test:')
auc, accuracy, TPR, FNR, recall, precision, specificity, prevalence, f1 = report(y_test, y_test_preds, thresh)
print('auc:%.3f'%auc,'accuracy:%.3f'%accuracy)
print('TPR:%.3f'%TPR,'FNR:%.3f'%FNR)
print('recall:%.3f'%recall)
print('precision:%.3f'%precision)
print('specificity:%.3f'%specificity)
print('prevalence:%.3f'%prevalence)
print('f1:%.3f'%f1)
print('  ')

Training:
auc:0.669 accuracy:0.619
TPR:0.551 FNR:0.449
recall:0.551
precision:0.638
specificity:0.687
prevalence:0.500
f1:0.592
  
Test:
auc:0.665 accuracy:0.673
TPR:0.543 FNR:0.457
recall:0.543
precision:0.184
specificity:0.689
prevalence:0.114
f1:0.275
  


# 7 Conclusion

Through this project, we created a binary classifier to predict the probability that a patient with diabetes would be readmitted to the hospital within 30 days. On held out test data, our best model had an AUC of of 0.67. Using this model, we are able to catch 54% of the readmissions from our model that performs approximately 1.5 times better than randomly selecting patients.